In [0]:
# 00_config_auth — autenticação via SAS guardado no Secret Scope (job-safe)

# --- parâmetros (podem ser sobrescritos pelo Job via "Parameters") ---
# Se preferir, edite aqui os valores padrão.
try:
    dbutils.widgets.get               # garante que widgets existem
except:
    pass

def _wget(name, default):
    try:
        dbutils.widgets.text(name, default)
    except:
        # widget já existia
        pass
    return dbutils.widgets.get(name)

acc   = _wget("storage_account", "stgflowdev001")        # sem .dfs
scope = _wget("secret_scope",    "meu_scope")
key   = _wget("secret_key",      "sas-stgflowdev001")

# --- lê o SAS do Secret Scope ---
try:
    sas_raw = dbutils.secrets.get(scope, key)
except Exception as e:
    raise Exception(
        f"[Auth] Falha ao ler secret: scope='{scope}', key='{key}'. "
        "Verifique se o scope e a chave existem e se você tem permissão."
    ) from e

# --- limpa o SAS (remove quebras de linha/espaços e '?' inicial, se houver) ---
sas = sas_raw.strip().replace("\n", "").replace("\r", "")
if sas.startswith("?"):
    sas = sas[1:]

if not sas.startswith("sv="):
    raise ValueError(
        "[Auth] O valor do SAS não parece válido (não inicia com 'sv='). "
        "Confira o segredo salvo no scope."
    )

# --- configura o Spark para usar SAS ---
spark.conf.set(f"fs.azure.account.auth.type.{acc}.dfs.core.windows.net", "SAS")
spark.conf.set(f"fs.azure.sas.token.provider.type.{acc}.dfs.core.windows.net",
               "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
spark.conf.set(f"fs.azure.sas.fixed.token.{acc}.dfs.core.windows.net", sas)

# --- caminhos base (para usar nas células seguintes) ---
bronze_base = f"abfss://bronze@{acc}.dfs.core.windows.net"
silver_base = f"abfss://silver@{acc}.dfs.core.windows.net"
gold_base   = f"abfss://gold@{acc}.dfs.core.windows.net"

print("[Auth] OK — SAS aplicado e caminhos definidos.")


In [0]:
%sql
-- lêr dados curados do Silver
CREATE OR REPLACE TEMP VIEW silver_fx AS
SELECT *
FROM delta.`abfss://silver@stgflowdev001.dfs.core.windows.net/fx_usdbrl/delta`;


In [0]:
%sql
CREATE OR REPLACE TEMP VIEW gold_fx_stage AS
WITH base AS (
  SELECT
    ref_date,
    base,
    symbol,
    CAST(rate AS DOUBLE) AS rate,
    provider,
    ingestion_ts
  FROM silver_fx
),
win AS (
  SELECT
    *,
    LAG(rate) OVER (ORDER BY ref_date)                                         AS rate_prev,
    AVG(rate) OVER (ORDER BY ref_date ROWS BETWEEN 6  PRECEDING AND CURRENT ROW) AS ma7,
    AVG(rate) OVER (ORDER BY ref_date ROWS BETWEEN 29 PRECEDING AND CURRENT ROW) AS ma30
  FROM base
)
SELECT
  ref_date,
  base,
  symbol,
  rate,
  rate_prev,
  (rate - rate_prev) AS rate_d1_change,
  ma7,
  ma30,
  provider,
  ingestion_ts,
  CURRENT_TIMESTAMP() AS snapshot_ts,
  YEAR(ref_date)      AS year,
  MONTH(ref_date)     AS month
FROM win;


In [0]:
%sql
CREATE TABLE IF NOT EXISTS delta.`abfss://gold@stgflowdev001.dfs.core.windows.net/fx_usdbrl/metrics_delta`
(
  ref_date        DATE,
  base            STRING,
  symbol          STRING,
  rate            DOUBLE,
  rate_prev       DOUBLE,
  rate_d1_change  DOUBLE,
  ma7             DOUBLE,
  ma30            DOUBLE,
  provider        STRING,
  ingestion_ts    TIMESTAMP,
  snapshot_ts     TIMESTAMP,
  year            INT,
  month           INT
)
USING delta
PARTITIONED BY (year, month);


In [0]:
%sql
MERGE INTO delta.`abfss://gold@stgflowdev001.dfs.core.windows.net/fx_usdbrl/metrics_delta` AS t
USING gold_fx_stage AS s
ON  t.ref_date = s.ref_date
AND t.base     = s.base
AND t.symbol   = s.symbol
WHEN MATCHED THEN UPDATE SET
  t.rate           = s.rate,
  t.rate_prev      = s.rate_prev,
  t.rate_d1_change = s.rate_d1_change,
  t.ma7            = s.ma7,
  t.ma30           = s.ma30,
  t.provider       = s.provider,
  t.ingestion_ts   = s.ingestion_ts,
  t.snapshot_ts    = s.snapshot_ts,
  t.year           = s.year,
  t.month          = s.month
WHEN NOT MATCHED THEN INSERT *
;


In [0]:
%sql
SELECT ref_date, base, symbol, rate, rate_prev, rate_d1_change, ma7, ma30, provider
FROM delta.`abfss://gold@stgflowdev001.dfs.core.windows.net/fx_usdbrl/metrics_delta`
ORDER BY ref_date DESC
LIMIT 50;


Databricks visualization. Run in Databricks to view.